In [1]:
import requests
import websocket
import json
from websocket import create_connection
import time
import pandas as pd
import numpy as np
from scipy.stats import norm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)
from IPython.display import clear_output
import random
from scipy.optimize import fsolve

# from VanillasFunctions import add,barrier_check

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [15]:
# Volatility index parameters such as vol, displayed decimals
assets=pd.read_csv("assets copy.csv")
#API URL
master_URL = "wss://qa141.deriv.dev/websockets/v3?app_id=1007&l=EN&brand=deriv"
#api token
api_token = "UIwxZvHVX2UYuet"

In [3]:
def barrier_check (symbol, duration, duration_unit, contract_type): 
    if duration_unit == "d":    
        barrier = "1000"        
    else:        
        barrier = "+0"        
    return_json = []
    def on_open(ws):        
        msg = {
            "proposal": 1,
            "amount": 60,
            "barrier": barrier,
            "basis": "stake",
            "contract_type": contract_type,
            "currency": "USD",
            "duration": duration,
            "duration_unit": duration_unit,
            "symbol": symbol
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):        
        amount = 60    
        message = json.loads(message)        
        print(message)        
        if 'error' in message.keys():            
            barrier = message['error']['details']['barrier_choices'][int(len(message['error']['details']['barrier_choices'])/2)]                
            if message['error']['details']['field'] == 'amount':                
                amount = float(message['error']['details']['max_stake'])
                print('amount =',amount)
                print(barrier)
                print(contract_type)
                print(duration)
                print(duration_unit)
                print(symbol)
            msg = {
                "proposal": 1,
                "amount": amount,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "USD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print('sent')
            ws.send(json.dumps(msg))              
        else:
            return_json.append(message)
            ws.close()            
    def on_close(ws, close_status_code, close_msg):
        print()             
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    barriers = return_json[-1]['proposal']['barrier_choices']
    return return_json[-1]

In [4]:
def calc_barriers (pd_series): #(spot, symbol, duration, duration_unit):    
    spot = float(pd_series.Spot)    
    symbol = pd_series.Asset    
    duration = pd_series.Start_expiry    
    duration_unit = pd_series.Duration[-1]    
    digits = int( assets[assets['Asset']== symbol].digits )    
    sigma = float( assets[assets['Asset']==symbol].IV )      
    n_digits = round( - np.log10(spot * sigma * np.sqrt(60/365/86400)) , 0) - 1    
    spot = round(spot/10**-n_digits)*10**-n_digits    
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400           
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400        
    if duration_unit != 'd':        
        deltas = np.arange(1,10,2)/10    
        real_strikes = spot * np.exp( (sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(deltas)) )    
        displayed_barriers = np.where( deltas > 0.5, np.round((real_strikes - spot)*10**digits)/10**digits , np.round((real_strikes - spot)*10**digits)/10**digits )        
    else:        
        temp_barriers = []        
        displayed_barriers = []
        central_strike = round( spot / (10 ** -n_digits) ) * (10 ** - n_digits )
        max_strike =  np.floor( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.1)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )
        min_strike =  np.ceil( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.9)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )
        strike_step_1 = round( (central_strike - min_strike) / 5 / 10 ) * 10
        strike_step_2 = round( (max_strike - central_strike) / 5 / 10 ) * 10 
        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1*i)
        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2*i)
        temp_barriers.append(max_strike)
        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)
    return displayed_barriers        

In [5]:
def calc_ppp_ask (pd_series, t_step, s_step):
    symbol = pd_series.Asset    
    duration = pd_series.Start_expiry    
    duration_unit = pd_series.Duration[-1]    
    stake = pd_series.Stake
    vol_charge = 1.025 
    t = ( int(duration[1]) - int(duration[0]) + t_step ) / 365 / 86400      
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        print('rounded')
        t -= 2 / 365 / 86400
    r = pd_series.Discount_rate     
    contract_type = pd_series.Contract_type
    number_of_contracts = pd_series.API_PPP        
    if s_step == 0:        
        spot = float(pd_series.Spot)
        strike = pd_series.Barrier    
    else:        
        spot = float(tick_history(int(duration[0]), symbol, s_step))        
        if duration_unit != 'd':            
            strike = spot + float(pd_series.Displayed_barrier)            
        else:            
            strike = pd_series.Barrier        
    sigma = float( assets[assets['Asset']==symbol].IV )    
    if duration_unit != 'd':        
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )        
    else:        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))    
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2        
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2        
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))    
    print(symbol,'s', spot, 'k',strike, r, t, vol,  phi,  'p',number_of_contracts, stake, 'd', (norm.cdf(phi*d1)))    
    return stake / ( bs_price + delta_charge + bs_markup )

In [6]:
def tick_history (tick,symbol,t):    
    if 'R' in symbol:
        r = 2
    else:
        r = 1        
    temp_json = []    
    def on_open(ws):
        omsg = {
          "ticks_history": symbol,
          "adjust_start_time": 1,
          "count": 5,
          "end": tick+t*r,
          "start": 1,
          "style": "ticks"
        }
        ws.send(json.dumps(omsg))        
    def on_message(ws, message):
        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            ws.send(json.dumps(msg))            
        else:        
            temp_json.append(message)
            ws.close()    
    def on_close(ws, close_status_code, close_msg):
        print()        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    return temp_json[-1]['history']['prices'][-1]    

In [7]:
def get_proposal (contract_id):     
    contract_id = int(contract_id)    
    return_json = []    
    def on_open(ws):        
        msg = {
            "authorize": api_token,
            "req_id": 1
        }        
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        print(message)
        if 'authorize' in message:            
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_id
            }            
            ws.send(json.dumps(msg))            
        else:                    
            return_json.append(message)            
            ws.close()
    def on_close(ws, close_status_code, close_msg):
        print()
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    return return_json[-1]

In [8]:
def get_barriers (symbol, duration, duration_unit, contract_type):     
    barrier = 0    
    if duration_unit == "d":
        barrier = "10000"
    else:
        barrier = "+0"        
    return_json = []
    def on_open(ws):        
        msg = {
            "proposal": 1,
            "amount": 60,
            "barrier": barrier,
            "basis": "stake",
            "contract_type": contract_type,
            "currency": "USD",
            "duration": duration,
            "duration_unit": duration_unit,
            "symbol": symbol
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        if duration_unit == "d":
            barrier = "10000"
        else:
            barrier = "+0"        
        print('\n',message)
        amount = 60
        message = json.loads(message)        
        if 'error' in message.keys():            
            if message['error']['details']['field'] == 'barrier':                
                barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]                
            elif message['error']['details']['field'] == 'amount':
                amount = int(message['error']['details']['min_stake']) + 5
                barrier = message['echo_req']['barrier']
                if message['error']['details']['min_stake'] > 2000:
                    barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                    amount = 2            
            print(amount)
            print(barrier)
            print(contract_type)
            print(duration)
            print(duration_unit)
            print(symbol)
            msg = {
                "proposal": 1,
                "amount": amount,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "USD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print(msg)
            ws.send(json.dumps(msg))              
        else:
            return_json.append(message)
            ws.close()            
    def on_close(ws, close_status_code, close_msg):
        print()              
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    barriers = return_json[-1]['proposal']['barrier_choices']    
    if 'qa' in master_URL:        
        return barriers, return_json[-1]['proposal']['pricing_args']['discount_rate']    
    else:        
        return barriers, 0

In [18]:
def get_contract (symbol, duration, duration_unit, contract_type): 
#     temp_unit = [duration_unit]
    return_json = []
    return_barrier = []    
    return_discount_rate = []
    contract_ID = []    
    def on_open(ws):        
        msg = {
            "authorize": api_token,
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        print('\n',message)
        barrier = 0        
        amount = 60        
        message = json.loads(message)        
        print('\n',message)
#         print('here1')
        if 'authorize' in message:
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "USD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            ws.send(json.dumps(msg))    
        elif 'error' in message.keys():
            if 'code' in message['error'].keys():
                print('here')
                print(message['error']['message'])
                print(message['error']['message'].split()[-1][:-1])
                print(amount)
                amount = float(message['error']['message'].split()[-1][:-1])                 
                if 'ax' in message['error']['message']:
                    amount -= 5
                else: 
                    amount += 5                    
                print('here',amount)
            print('here2',amount)    
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])            
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "USD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            print('msgcre')
            print(msg)
            ws.send(json.dumps(msg))
            print('sent')
        elif 'buy' in message:
            print('bought')
            contract_ID.append(message['buy']['contract_id'])
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_ID[-1]
            }
            ws.send(json.dumps(msg))            
        else:
            return_json.append(message)            
            ws.close()                                          
    def on_close(ws, close_status_code, close_msg):
        print()
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()   
    return return_json[-1], return_discount_rate[-1], return_barrier[-1]

In [10]:
def calc_ppp_bid (proposal, t_step, s_step):
    symbol = proposal['proposal_open_contract']['underlying']    
    stake = float(proposal['proposal_open_contract']['bid_price'])
    vol_charge = 0.975 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['current_spot_time'] + t_step ) / 365 / 86400          
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400       
    if t > 1 / 365:        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    else:        
        r = 0                
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )    
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])               
    if s_step == 0:        
        spot = float(proposal['proposal_open_contract']['current_spot'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(int(proposal['proposal_open_contract']['current_spot_time']), symbol, s_step))        
        if t > 1 / 365:        
            strike = float(proposal['proposal_open_contract']['barrier'])
        else:
            strike = spot + float(proposal['proposal_open_contract']['barrier']) - float(proposal['proposal_open_contract']['current_spot'])    
    sigma = float( assets[assets['Asset']==symbol].IV )        
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2    
    print(spot,strike,r, t, phi,  number_of_contracts, stake)
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))
    return ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup) * number_of_contracts    

In [11]:
def sell_contract(contract_id):    
    contract_id = int(contract_id)    
    temp_json = []    
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))        
    def on_message(ws, message):
        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:            
            msg = {
              "sell": contract_id,
              "price": 0
            }
            ws.send(json.dumps(msg))            
        else:        
            temp_json.append(float(message['sell']['sold_for']))
            ws.close()    
    def on_close(ws, close_status_code, close_msg):
        print()        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    return temp_json[-1]

In [12]:
def calc_ppp_sold (proposal, t_step, s_step):
    symbol = proposal['proposal_open_contract']['underlying']    
    stake = float(proposal['proposal_open_contract']['sell_price'])
    vol_charge = 0.975 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['exit_tick_time'] + t_step) / 365 / 86400  
    if t > 1 / 365:        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    else:        
        r = 0                
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )    
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])        
    if s_step == 0:        
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))        
        strike = float(proposal['proposal_open_contract']['barrier'])    
    sigma = float( assets[assets['Asset']==symbol].IV )
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2            
    print('s',spot,'k',strike,'r',r,'t', t, phi,'n',  number_of_contracts,'s', stake)
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))    
    price = ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup)
    print(price)
    return price * number_of_contracts

In [13]:
def calc_ppp_exp (proposal, t_step, s_step):    
    stake = float(proposal['proposal_open_contract']['sell_price'])    
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])        
    if s_step == 0:        
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))        
        strike = float(proposal['proposal_open_contract']['barrier'])    
    if contract_type == "VANILLALONGCALL":
        phi = 1
    elif  contract_type == "VANILLALONGPUT":
        phi = -1                
    print(spot,strike, phi,  number_of_contracts, stake)
    return max( (spot - strike) * phi * number_of_contracts, 0)    

In [93]:
# barrier_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Duration', 'Start_expiry', 'API_barrier_choice', 'Calc_barrier_choices', 'json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    clear_output(wait = True)    
    print(len(barrier_df))
    if len(barrier_df) > 100 - 1:
        break                
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':        
        max_duration = 1440
    elif duration_unit == 'h':        
        max_duration = 24
    elif duration_unit == 'd':       
        max_duration = 365
    i = np.random.randint(5)    
    if i < 2:        
        duration = max_duration**i        
    else:        
        duration = np.random.randint(1,max_duration)        
    price_proposal = barrier_check(asset, duration, duration_unit, contract_type)    
    barrier_df.loc[len(barrier_df)] = [ price_proposal['echo_req']['symbol'], price_proposal['echo_req']['contract_type'], price_proposal['proposal']['spot'], \
                                        price_proposal['proposal']['spot_time'], str(price_proposal['echo_req']['duration']) + price_proposal['echo_req']['duration_unit'], \
                                        [ price_proposal['proposal']['date_start'], price_proposal['proposal']['date_expiry'] ], \
                                        [float(x) for x in price_proposal['proposal']['barrier_choices']], 0, price_proposal ]    
    time.sleep(0.3)
clear_output(wait = True)
barrier_df

,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json
0,R_10,VANILLALONGCALL,10019.448,1687423390,963m,"[1687423390, 1687481170]","[-54.72, -22.37, 0.09, 22.61, 55.21]","[55.21, 22.61, 0.09, -22.37, -54.72]","{'echo_req': {'amount': 60, 'barrier': '+0.090', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'USD', 'duration': 963, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_10'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '60.00', 'barrier_choices'..."
1,1HZ10V,VANILLALONGPUT,10057.62,1687423392,1440m,"[1687423392, 1687509792]","[-67.1, -27.4, 0.1, 27.8, 67.8]","[67.8, 27.8, 0.1, -27.4, -67.1]","{'echo_req': {'amount': 60, 'barrier': '+0.10', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 1440, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '60.00', 'barrier_choices..."
2,1HZ25V,VANILLALONGCALL,9996.07,1687423394,365d,"[1687423394, 1719014399]","[7500.0, 8000.0, 8500.0, 9000.0, 9500.0, 10000.0, 10840.0, 11680.0, 12520.0, 13360.0, 14200.0]","[7500.0, 8000.0, 8500.0, 9000.0, 9500.0, 10000.0, 10840.0, 11680.0, 12520.0, 13360.0, 14200.0]","{'echo_req': {'amount': 60, 'barrier': '10000.00', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'USD', 'duration': 365, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ25V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '60.00', 'barrier_choi..."
3,R_10,VANILLALONGPUT,10019.780,1687423396,1440m,"[1687423396, 1687509796]","[-66.85, -27.33, 0.14, 27.68, 67.58]","[67.58, 27.68, 0.14, -27.33, -66.85]","{'echo_req': {'amount': 60, 'barrier': '+0.140', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 1440, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_10'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '60.00', 'barrier_choices'..."
4,1HZ10V,VANILLALONGPUT,10057.64,1687423398,365d,"[1687423398, 1719014399]","[8900.0, 9130.0, 9360.0, 9590.0, 9820.0, 10060.0, 10350.0, 10640.0, 10930.0, 11220.0, 11490.0]","[8900.0, 9130.0, 9360.0, 9590.0, 9820.0, 10060.0, 10350.0, 10640.0, 10930.0, 11220.0, 11490.0]","{'echo_req': {'amount': 60, 'barrier': '10060.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 365, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '60.00', 'barrier_choic..."
...,...,...,...,...,...,...,...,...,...
95,R_50,VANILLALONGPUT,9814.1413,1687424440,235d,"[1687424441, 1707782399]","[6400.0, 7080.0, 7760.0, 8440.0, 9120.0, 9800.0, 11380.0, 12960.0, 14540.0, 16120.0, 17700.0]",0,"{'echo_req': {'amount': 60, 'barrier': '9800.0000', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 235, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_50'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '60.00', 'barrier_choice..."
96,R_10,VANILLALONGCALL,10017.767,1687424442,1m,"[1687424443, 1687424503]","[-1.74, -0.71, 0.0, 0.71, 1.74]",0,"{'echo_req': {'amount': 60, 'barrier': '+0.000', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'USD', 'duration': 1, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_10'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '60.00', 'barrier_choices': ..."
97,1HZ10V,VANILLALONGPUT,10057.27,1687424444,281d,"[1687424445, 1711756799]","[9030.0, 9240.0, 9450.0, 9660.0, 9870.0, 10060.0, 10310.0, 10560.0, 10810.0, 11060.0, 11300.0]",0,"{'echo_req': {'amount': 60, 'barrier': '10060.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 281, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '60.00', 'barri

In [94]:
Calc_barrier_choices = [] 
for i in barrier_df.index:    
    Calc_barrier_choices.append( calc_barriers(barrier_df.iloc[i]) )
barrier_df.Calc_barrier_choices = Calc_barrier_choices
print(len(barrier_df), 'Test cases','\nMismatches =',\
      len(barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]),\
      '\nMismatches listed below :')
barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]

100 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json


In [25]:
# PPP_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Calc_PPP','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    clear_output(wait=True)
    print(len(PPP_df))   
    if len(PPP_df)>100-1:
        break        
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)    
    if i<2:        
        duration = max_duration**i        
    else:        
        duration = np.random.randint(1, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    PPP_df.loc[len(PPP_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, proposal]
clear_output(wait=True)
PPP_df

,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
0,1HZ75V,VANILLALONGCALL,9680.2800,1687416379,60.0,120d,11460.00,11460.0000,"[1687416379, 1697846399]",1339,0.056249,0.05608,0.056082,"{'echo_req': {'contract_id': 1339, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '308', 'barrier': '11460.00', 'barrier_count': 1, 'bid_price': 55.48, 'buy_price': '60.00', 'contract_id': '1339', 'contract_type': 'VANILLALONGCALL', '..."
1,1HZ100V,VANILLALONGPUT,9930.1000,1687416432,60.0,21h,-239.40,9690.7000,"[1687416432, 1687492032]",1379,0.000000,0.60324,0.603242,"{'echo_req': {'contract_id': 1379, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '308', 'barrier': '9690.70', 'barrier_count': 1, 'bid_price': 54.68, 'buy_price': '60.00', 'contract_id': '1379', 'contract_type': 'VANILLALONGPUT', 'cu..."
2,R_10,VANILLALONGCALL,10005.6040,1687416438,60.0,12h,+0.070,10005.6740,"[1687416439, 1687459639]",1399,0.000000,3.97135,3.971352,"{'echo_req': {'contract_id': 1399, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '308', 'barrier': '10005.674', 'barrier_count': 1, 'bid_price': 57.07, 'buy_price': '60.00', 'contract_id': '1399', 'contract_type': 'VANILLALONGCALL', ..."
3,R_50,VANILLALONGCALL,9800.4796,1687416448,60.0,1440m,-319.9920,9480.4876,"[1687416448, 1687502848]",1419,0.000000,0.17995,0.179954,"{'echo_req': {'contract_id': 1419, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '308', 'barrier': '9480.4876', 'barrier_count': 1, 'bid_price': 59.59, 'buy_price': '60.00', 'contract_id': '1419', 'contract_type': 'VANILLALONGCALL', ..."
4,1HZ50V,VANILLALONGCALL,10211.4200,1687416452,60.0,365d,6100.00,6100.0000,"[1687416452, 1719014399]",1439,0.055473,0.01429,0.014289,"{'echo_req': {'contract_id': 1439, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '308', 'barrier': '6100.00', 'barrier_count': 1, 'bid_price': 59.38, 'buy_price': '60.00', 'contract_id': '1439', 'contract_type': 'VANILLALONGCALL', 'c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1HZ100V,VANILLALONGPUT,9756.6900,1687420783,60.0,36d,7480.00,7480.0000,"[1687420783, 1690588799]",4639,0.051598,0.18807,0.000000,"{'echo_req': {'contract_id': 4639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '308', 'barrier': '7480.00', 'barrier_count': 1, 'bid_price': 52.98, 'buy_price': '60.00', 'contract_id': '4639', 'contract_type': 'VANILLALONGPUT', 'cu..."
96,1HZ75V,VANILLALONGCALL,9586.2300,1687420786,60.0,20h,+6.20,9592.4300,"[1687420786, 1687492786]",4679,0.000000,0.43657,0.000000,"{'echo_req': {'contract_id': 4679, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '308', 'barrier': '9592.43', 'barrier_count': 1, 'bid_price': 56.83, 'buy_price': '60.00', 'contract_id': '4679', 'contract_type': 'VANILLALONGCALL', 'c..."
97,1HZ100V,VANILLALONGPUT,9752.5200,1687420790,60.0,11h,-174.40,9578.1200,"[1687420790, 1687460390]",4739,0.000000,0.85986,0.000000,"{'echo_req': {'contract_id': 4739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '308', 'barrier': '9578.12', 'barrier_count': 1, 'bid_price': 54.84, 'buy_price': '60.00', 'contract_id': '4739', 'contract_type': 'VANILLALONGPUT', 'cu..."
98,1HZ75V,VANILLALONGPUT,9587.6000,1687420794,60.0,561m,-294.00,9293.6000,"[1687420794, 1687454454]",4759,0.000000,4.90066,0.000000,"{'echo_req': {'contract_id': 4759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '308', 'barrier': '9293.60', 'barrier_count': 1, 'bid_price': 4

In [26]:
Calc_ppp = [] 
for i in PPP_df.index:   
    Calc_ppp.append( calc_ppp_ask(PPP_df.iloc[i], 0, 0) )
PPP_df.Calc_PPP = Calc_ppp
clear_output(wait = True)
print(len(PPP_df), 'Test cases','\nMismatches =',\
      len(PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,5) != 0]),\
      '\nMismatches listed below :')
df = PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,5) != 0]
df

100 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json


In [35]:
# Bid_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:   
    print(len(Bid_df))    
    if len(Bid_df)>100-1:
        break            
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)        
    duration = np.random.randint(max_duration/2, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    Bid_df.loc[len(Bid_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
                               
    clear_output(wait=True)
Bid_df

100


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ75V,VANILLALONGCALL,9650.6000,1687422855,360.5,736m,+359.20,10009.8000,"[1687422855, 1687467015]",7599,0.000000,26.30536,1687422865,301.29,301.287520,"{'echo_req': {'contract_id': 7599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10009.80', 'barrier_count': 1, 'bid_price': 301.29, 'buy_price': '360.50', 'contract_id': '7599', 'contract_type': 'VANILLALONGCALL',...","{'echo_req': {'contract_id': 7599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10009.80', 'barrier_count': 1, 'bid_price': 298.52, 'buy_price': '360.50', 'contract_id': '7599', 'contract_type': 'VANILLALONGCALL',..."
1,R_50,VANILLALONGPUT,9854.2769,1687422858,60.0,22h,-126.1660,9728.1109,"[1687422859, 1687502059]",7619,0.000000,1.20958,1687422866,55.20,55.198309,"{'echo_req': {'contract_id': 7619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '9728.1109', 'barrier_count': 1, 'bid_price': '55.20', 'buy_price': '60.00', 'contract_id': '7619', 'contract_type': 'VANILLALONGPUT',...","{'echo_req': {'contract_id': 7619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '9728.1109', 'barrier_count': 1, 'bid_price': 54.81, 'buy_price': '60.00', 'contract_id': '7619', 'contract_type': 'VANILLALONGPUT', '..."
2,1HZ100V,VANILLALONGPUT,9656.1800,1687422885,60.0,16h,-206.30,9449.8800,"[1687422885, 1687480485]",7639,0.000000,0.71713,0,0.00,0.000000,0,"{'echo_req': {'contract_id': 7639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '9449.88', 'barrier_count': 1, 'bid_price': 54.86, 'buy_price': '60.00', 'contract_id': '7639', 'contract_type': 'VANILLALONGPUT', 'cu..."
3,1HZ50V,VANILLALONGCALL,10366.3900,1687422889,60.0,18h,+2.70,10369.0900,"[1687422889, 1687487689]",7659,0.000000,0.63330,0,0.00,0.000000,0,"{'echo_req': {'contract_id': 7659, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10369.09', 'barrier_count': 1, 'bid_price': 57.03, 'buy_price': '60.00', 'contract_id': '7659', 'contract_type': 'VANILLALONGCALL', '..."
4,R_75,VANILLALONGCALL,9370.4814,1687422892,60.0,322d,21580.0000,21580.0000,"[1687422893, 1715299199]",7679,0.055701,0.10399,0,0.00,0.000000,0,"{'echo_req': {'contract_id': 7679, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '21580.0000', 'barrier_count': 1, 'bid_price': 50.76, 'buy_price': '60.00', 'contract_id': '7679', 'contract_type': 'VANILLALONGCALL',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,R_25,VANILLALONGCALL,10118.6240,1687423250,60.0,15h,-132.490,9986.1340,"[1687423250, 1687477250]",9499,0.000000,0.43490,0,0.00,0.000000,0,"{'echo_req': {'contract_id': 9499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '9986.134', 'barrier_count': 1, 'bid_price': '59.60', 'buy_price': '60.00', 'contract_id': '9499', 'contract_type': 'VANILLALONGCALL',..."
96,1HZ25V,VANILLALONGPUT,9993.8600,1687423253,60.0,16h,+0.60,9994.4600,"[1687423253, 1687480853]",9519,0.000000,1.36472,0,0.00,0.000000,0,"{'echo_req': {'contract_id': 9519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '9994.46', 'barrier_count': 1, 'bid_price': 57.09, 'buy_price': '60.00', 'contract_id': '9519', 'contract_type': 'VANILLALONGPUT', 'cu..."
97,R_100,VANILLALONGPUT,10399.8100,1687423256,60.0

In [36]:
calc_bid = [] 
bid_time = []
api_bid = []
bid_prop = []
for i in Bid_df.index:    
    if i >= len(bid_time):        
        clear_output(wait = True)
        proposal = get_proposal(Bid_df.loc[i].ID)
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, 0, 0) )        
        bid_prop.append(proposal)    
Bid_df.Bid_time = bid_time
Bid_df.API_bid = api_bid
Bid_df.Calc_bid = calc_bid
Bid_df.bid_prop = bid_prop
clear_output(wait = True)
print(len(Bid_df), 'Test cases','\nMismatches =',\
      len(Bid_df[(Bid_df.API_bid) - np.round(Bid_df.Calc_bid,2) != 0]),\
      '\nMismatches listed below :')
df = Bid_df[(Bid_df.API_bid) - np.round(Bid_df.Calc_bid,2) != 0]
df

100 Test cases 
Mismatches = 2 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
36,R_100,VANILLALONGPUT,10386.950,1687423016,60.0,12h,-192.90,10194.050,"[1687423017, 1687466217]",8319,0.0,0.77006,1687423352,52.2,52.205303,"{'echo_req': {'contract_id': 8319, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10194.05', 'barrier_count': 1, 'bid_price': '52.20', 'buy_price': '60.00', 'contract_id': '8319', 'contract_type': 'VANILLALONGPUT', ...","{'echo_req': {'contract_id': 8319, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10194.05', 'barrier_count': 1, 'bid_price': 54.85, 'buy_price': '60.00', 'contract_id': '8319', 'contract_type': 'VANILLALONGPUT', 'c..."
90,R_10,VANILLALONGCALL,10020.675,1687423230,60.0,16h,+55.120,10075.795,"[1687423231, 1687480831]",9399,0.0,27.07259,1687423446,51.8,51.805847,"{'echo_req': {'contract_id': 9399, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10075.795', 'barrier_count': 1, 'bid_price': '51.80', 'buy_price': '60.00', 'contract_id': '9399', 'contract_type': 'VANILLALONGCALL'...","{'echo_req': {'contract_id': 9399, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10075.795', 'barrier_count': 1, 'bid_price': 49.83, 'buy_price': '60.00', 'contract_id': '9399', 'contract_type': 'VANILLALONGCALL', ..."


In [40]:
# Minus 1 second from duration to match remaining 2 cases
calc_bid = [] 
bid_time = []
api_bid = []
bid_prop = []
for i in df.index:   
    if i >= len(bid_time):       
        clear_output(wait = True)
        proposal = df.loc[i].bid_prop
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, -1,0) )        
        bid_prop.append(proposal)    
df.Bid_time = bid_time
df.API_bid = api_bid
df.Calc_bid = calc_bid
df.bid_prop = bid_prop
clear_output(wait = True)
print(len(df), 'Test cases','\nMismatches =',\
      len(df[(df.API_bid) - np.round(df.Calc_bid,2) != 0]),\
      '\nMismatches listed below :')
df2 = df[(df.API_bid) - np.round(df.Calc_bid,2) != 0]
df2

2 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json


In [29]:
# Sold_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:   
    print(len(Sold_df))    
    if len(Sold_df) > 100 - 1:
        break    
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(['m','h'])
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)        
    duration = np.random.randint(max_duration/2, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    Sold_df.loc[len(Sold_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
                               
    clear_output(wait=True)    
Sold_df

100


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ10V,VANILLALONGCALL,10070.4300,1687421303,60.0,1001m,-22.90,10047.5300,"[1687421303, 1687481363]",5359,0,1.89416,0,0,0,0,"{'echo_req': {'contract_id': 5359, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10047.53', 'barrier_count': 1, 'bid_price': 58.55, 'buy_price': '60.00', 'contract_id': '5359', 'contract_type': 'VANILLALONGCALL', '..."
1,R_100,VANILLALONGCALL,10253.6700,1687421306,60.0,1182m,-596.40,9657.2700,"[1687421307, 1687492227]",5379,0,0.09650,0,0,0,0,"{'echo_req': {'contract_id': 5379, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '9657.27', 'barrier_count': 1, 'bid_price': 59.59, 'buy_price': '60.00', 'contract_id': '5379', 'contract_type': 'VANILLALONGCALL', 'c..."
2,R_75,VANILLALONGCALL,9422.3747,1687421310,60.0,1190m,+183.6700,9606.0447,"[1687421310, 1687492710]",5399,0,0.91057,0,0,0,0,"{'echo_req': {'contract_id': 5399, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '9606.0447', 'barrier_count': 1, 'bid_price': 54.67, 'buy_price': '60.00', 'contract_id': '5399', 'contract_type': 'VANILLALONGCALL', ..."
3,1HZ25V,VANILLALONGPUT,9979.3000,1687421314,60.0,22h,+162.70,10142.0000,"[1687421314, 1687500514]",5419,0,0.35480,0,0,0,0,"{'echo_req': {'contract_id': 5419, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10142.00', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': '60.00', 'contract_id': '5419', 'contract_type': 'VANILLALONGPUT', 'c..."
4,1HZ100V,VANILLALONGPUT,9702.7200,1687421318,60.0,1155m,-555.50,9147.2200,"[1687421318, 1687490618]",5439,0,2.49827,0,0,0,0,"{'echo_req': {'contract_id': 5439, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '9147.22', 'barrier_count': 1, 'bid_price': 50.03, 'buy_price': '60.00', 'contract_id': '5439', 'contract_type': 'VANILLALONGPUT', 'cu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,R_50,VANILLALONGCALL,9831.7154,1687421662,60.0,724m,+237.5850,10069.3004,"[1687421663, 1687465103]",7259,0,6.33751,0,0,0,0,"{'echo_req': {'contract_id': 7259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10069.3004', 'barrier_count': 1, 'bid_price': '49.80', 'buy_price': '60.00', 'contract_id': '7259', 'contract_type': 'VANILLALONGCALL..."
96,1HZ25V,VANILLALONGCALL,9990.2400,1687421667,60.0,22h,+66.70,10056.9400,"[1687421667, 1687500867]",7279,0,2.42469,0,0,0,0,"{'echo_req': {'contract_id': 7279, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10056.94', 'barrier_count': 1, 'bid_price': 54.73, 'buy_price': '60.00', 'contract_id': '7279', 'contract_type': 'VANILLALONGCALL', '..."
97,1HZ25V,VANILLALONGCALL,9990.5500,1687421671,60.0,18h,-143.50,9847.0500,"[1687421671, 1687486471]",7299,0,0.40165,0,0,0,0,"{'echo_req': {'contract_id': 7299, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '9847.05', 'barrier_count': 1, 'bid_price': '59.60', 'buy_price': '60.00', 'contract_id': '7299', 'contract_type': 'VANILLALONGCALL', ..."
98,R_50,VANILLALONGPUT,9832.6571,1687421674,60.0,13h,+246.7830,10079.4401,"[1687421675, 1687468475]",7319,0,0.23382,0,0,0,0,"{'echo_req': {'contract_id': 7319, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10079.4401', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': '60.0

In [30]:
calc_sold = []
sold_time = []
api_sold = []
sold_prop = []
for i in Sold_df.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)        
        sell_contract(Sold_df.loc[i].ID)        
        proposal = get_proposal(Sold_df.loc[i].ID)
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, 0, 0) )        
        sold_prop.append(proposal)    
Sold_df['Sold_time'] = sold_time
Sold_df['API_sold'] = api_sold
Sold_df['Calc_sold'] = calc_sold
Sold_df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(Sold_df), 'Test cases','\nMismatches =',\
      len(Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,2) != 0]),\
      '\nMismatches listed below :')
df = Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,2) != 0]
df

100 Test cases 
Mismatches = 2 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,...,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json,Sold_time,API_sold,Calc_sold,Sold_prop
55,R_25,VANILLALONGCALL,10109.5210,1687421510,60.0,1417m,+170.290,10279.8110,"[1687421510, 1687506530]",6459,...,8.84513,0,0,0,0,"{'echo_req': {'contract_id': 6459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10279.811', 'barrier_count': 1, 'bid_price': '49.80', 'buy_price': '60.00', 'contract_id': '6459', 'contract_type': 'VANILLALONGCALL'...",1687422164,50.50,50.505699,"{'echo_req': {'contract_id': 6459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'audit_details': {'contract_start': [{'epoch': 1687421506, 'tick': '10107.789', 'tick_display_value': '10107.789'}, {'epoch': 1687421508, 'tick':..."
95,R_50,VANILLALONGCALL,9831.7154,1687421662,60.0,724m,+237.5850,10069.3004,"[1687421663, 1687465103]",7259,...,6.33751,0,0,0,0,"{'echo_req': {'contract_id': 7259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'barrier': '10069.3004', 'barrier_count': 1, 'bid_price': '49.80', 'buy_price': '60.00', 'contract_id': '7259', 'contract_type': 'VANILLALONGCALL...",1687422306,71.71,71.716525,"{'echo_req': {'contract_id': 7259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '328', 'audit_details': {'contract_start': [{'epoch': 1687421658, 'tick': '9832.3481', 'tick_display_value': '9832.3481'}, {'epoch': 1687421660, 'tick':..."


In [49]:
# Minus 1 second from duration to match remaining 2 cases
calc_sold = []
sold_time = []
api_sold = []
sold_prop = []
for i in df.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)        
        proposal = Sold_df.loc[55].Sold_prop
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, -1, 0) )        
        sold_prop.append(proposal)    
df['Sold_time'] = sold_time
df['API_sold'] = api_sold
df['Calc_sold'] = calc_sold
df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(df), 'Test cases','\nMismatches =',\
      len(df[(df.API_sold) - np.round(df.Calc_sold,2) != 0]),\
      '\nMismatches listed below :')
df2 = df[(df.API_sold) - np.round(df.Calc_sold,2) != 0]
df2

2 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,...,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json,Sold_time,API_sold,Calc_sold,Sold_prop


In [75]:
# Expired_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'API_exp', 'Calc_exp','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m')
while True:   
    print(len(Expired_df))   
    if len(Expired_df)> 100-1:
        break            
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    duration_unit = 'm'
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)        
    duration = np.random.randint(1, 45)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    Expired_df.loc[len(Expired_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, proposal]
    clear_output(wait=True)
Expired_df

100


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json
0,R_100,VANILLALONGCALL,10299.3200,1687420530,60.0,1h,-57.00,10242.3200,"[1687420531, 1687424131]",2019,0,0.75936,0,0,"{'echo_req': {'contract_id': 2019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '288', 'barrier': '10242.32', 'barrier_count': 1, 'bid_price': 58.55, 'buy_price': '60.00', 'contract_id': '2019', 'contract_type': 'VANILLALONGCALL', '..."
1,1HZ100V,VANILLALONGPUT,9823.6200,1687420534,60.0,1h,-132.70,9690.9200,"[1687420535, 1687424135]",2059,0,10.91360,0,0,"{'echo_req': {'contract_id': 2059, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '288', 'barrier': '9690.92', 'barrier_count': 1, 'bid_price': 47.69, 'buy_price': '60.00', 'contract_id': '2059', 'contract_type': 'VANILLALONGPUT', 'cu..."
2,1HZ50V,VANILLALONGCALL,10251.8000,1687420538,60.0,1h,+29.00,10280.8000,"[1687420538, 1687424138]",2099,0,5.53180,0,0,"{'echo_req': {'contract_id': 2099, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '288', 'barrier': '10280.80', 'barrier_count': 1, 'bid_price': 56.36, 'buy_price': '60.00', 'contract_id': '2099', 'contract_type': 'VANILLALONGCALL', '..."
3,R_75,VANILLALONGPUT,9365.0373,1687420542,60.0,1h,+39.8860,9404.9233,"[1687420542, 1687424142]",2139,0,1.09644,0,0,"{'echo_req': {'contract_id': 2139, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '288', 'barrier': '9404.9233', 'barrier_count': 1, 'bid_price': 58.57, 'buy_price': '60.00', 'contract_id': '2139', 'contract_type': 'VANILLALONGPUT', '..."
4,1HZ10V,VANILLALONGCALL,10068.5800,1687420546,60.0,1h,-5.60,10062.9800,"[1687420546, 1687424146]",2179,0,7.73864,0,0,"{'echo_req': {'contract_id': 2179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '288', 'barrier': '10062.98', 'barrier_count': 1, 'bid_price': 58.55, 'buy_price': '60.00', 'contract_id': '2179', 'contract_type': 'VANILLALONGCALL', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1HZ100V,VANILLALONGCALL,9659.3400,1687422011,60.0,17m,-70.30,9589.0400,"[1687422011, 1687423031]",7499,0,0.82057,0,0,"{'echo_req': {'contract_id': 7499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '288', 'barrier': '9589.04', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': '60.00', 'contract_id': '7499', 'contract_type': 'VANILLALONGCALL', 'c..."
96,R_75,VANILLALONGPUT,9478.9259,1687422014,60.0,3m,+8.8960,9487.8219,"[1687422015, 1687422195]",7519,0,4.89809,0,0,"{'echo_req': {'contract_id': 7519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '288', 'barrier': '9487.8219', 'barrier_count': 1, 'bid_price': 58.56, 'buy_price': '60.00', 'contract_id': '7519', 'contract_type': 'VANILLALONGPUT', '..."
97,R_25,VANILLALONGCALL,10114.6350,1687422018,60.0,3m,-7.680,10106.9550,"[1687422019, 1687422199]",7539,0,7.50667,0,0,"{'echo_req': {'contract_id': 7539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '288', 'barrier': '10106.955', 'barrier_count': 1, 'bid_price': '59.60', 'buy_price': '60.00', 'contract_id': '7539', 'contract_type': 'VANILLALONGCALL'..."
98,1HZ100V,VANILLALONGCALL,9660.5900,1687422022,60.0,31m,+96.20,9756.7900,"[1687422022, 1687423882]",7559,0,15.80652,0,0,"{'echo_req': {'contract_id': 7559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '288', 'barrier': '9756.79', 'barrier_count': 1, 'bid_price': 49.72, 'buy_price': '60.00', 'contract_id': '7559', 'contract_type': 'VANILLALONGCALL', 'c..."


In [95]:
calc_exp = [] 
exp_time = []
api_exp = []
for i in Expired_df.index:    
    if i >= len(exp_time):        
        clear_output(wait = True)                
       # getting proposals for expired contracts to save API payout
        proposal = get_proposal(Expired_df.loc[i].ID)        
        if proposal['proposal_open_contract']['is_sold'] == 1:            
            exp_time.append(proposal['proposal_open_contract']['date_expiry'])
            api_exp.append(float(proposal['proposal_open_contract']['sell_price']))
            # Calculating expired payout to compare againts api
            calc_exp.append( calc_ppp_exp(proposal, 0, 0) )            
        else:            
            exp_time.append(0)            
            api_exp.append(0)
            calc_exp.append(0)    
Expired_df['Exp_time'] = exp_time
Expired_df['API_exp'] = api_exp
Expired_df['Calc_exp'] = calc_exp
clear_output(wait = True)
print(len(Expired_df), 'Test cases','\nMismatches =',\
      len(Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,2) != 0]),\
      '\nMismatches listed below :')
df = Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,2) != 0]
df

100 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time
